<div style="text-align: right"><b> Ana Valentina López Chacón & Isabella Martínez Martínez & Sara Julieth Zuleta Quevedo. </b></div>

# **Proyecto 4: ASR y algo más**

En este proyecto la idea es que desarrollen al menos un sistema ASR, pero en el cual involucren otro tipo de procesamiento de NLP. Este proyecto es sólo de implementación, no requieren entrenar (pero podrían intentarlo).

## **Proyecto 1. Traducción simultánea**

En el mundo existen más de 7100 idiomas, una diversificación inmensa (y rica culturalmente hablando) que hace que en idiomas no tan comunes sean complicada la comunicación o el aprendizaje. El objetivo de este proyecto es primero aprender a implementar diversas etapas de ASR y también de TTS, pero involucrando un modulo de NLP.

### **Parte 1.**

Desarrollar un pipeline que consiste de tres etapas:

1.  Una etapa que convierta el voz a texto. Por ejemplo, que transcriba el español a texto. Recomiendo que usen un modelo [wav2vec2](https://huggingface.co/models?other=wav2vec2). Para español un modelo que funciona bien, es [este](https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-spanish).

2. A partir del texto transcrito, un modulo que realice una traducción de un idioma a otro. Por ejemplo, de español a inglés. Pueden usar un modelo multi-idioma de un transformer encoder-decoder como los basados en [MarianMT](https://huggingface.co/docs/transformers/main/en/model_doc/marian) de la universidad de Helsinki, por ejemplo [este modelo](https://huggingface.co/Helsinki-NLP/opus-mt-mul-en). O pueden usar [este otro modelo](https://huggingface.co/Helsinki-NLP/opus-mt-es-en), específico para pasar de español a inglés.

3. Implementar un modulo que convierta el texto a voz, mediante un modelo TTS. [Modelos para TTS](https://huggingface.co/models?pipeline_tag=text-to-speech&sort=downloads) hay muchos, y son relativamente rápidos. [Este modelo] de tipo [Fast2Speech 2](https://arxiv.org/abs/2006.04558), por ejemplo, realiza un TTS en inglés rápido.

En esta etapa la se puede construir el pipeline para que funcione en modo offline, es decir, se graba un audio y luego produce un archivo de audio. 

### **Parte 2. Pipeline en modo online**

En esta parte deberán colocar el pipeline desarrollado para funcionar en modo online, es decir, hacer que la traducción sea en tiempo real (en la medida de lo posible). El usuario habla y posteriormente en un tiempo pequeño se escucha la traducción. Se debe desarrollar una pequeña interfaz que haga esto (usando Gradio o Streamlit).

### **Parte 3. Multi-lenguaje**
Dado que se ha construido un pipeline, se puede cambiar de forma sencilla los idiomas. Sustituya los modelos para hacer una traducción diferente, por ejemplo de inglés a español, de español a fránces. Pruebe diferentes configuraciones. Si logran integran en la interfaz la configuración de idioma, se valorará en la calificación.


-----
Es importante que realicen gráficos y visualizaciones que ayuden a la interpretación. No olviden ir analizando y comentando los hallazgos, y sobretodo **concluir**. El entregable es un informe en un notebook de Jupyter, debidamente presentado y comentado.

# **Implementación**

Con el fin de obtener un resultado satisfactorio, es necesario instalar las siguientes dependencias.

In [1]:
!pip install urllib3==1.24.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sentencepiece -q
!pip install gradio -q
!pip install transformers -q
!pip install huggingsound -q
!pip install g2p-en -q
!pip install ffmpeg-python -q
!pip install phonemizer -q
!pip install hanziconv -q
!pip install g2pc -q

     |████████████████████████████████| 1.2 MB 7.5 MB/s 
     |████████████████████████████████| 5.1 MB 8.8 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 84 kB 3.6 MB/s 
     |████████████████████████████████| 253 kB 67.1 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 2.0 MB 69.8 MB/s 
     |████████████████████████████████| 1.1 MB 56.6 MB/s 
     |████████████████████████████████| 144 kB 66.8 MB/s 
     |████████████████████████████████| 271 kB 62.9 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 11.1 MB 55.6 MB/s 
     |████████████████████████████████| 80 kB 10.7 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 4.0 MB 54.1 MB/s 
     |██████████████████████

In [3]:
!apt-get install espeak

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
G

**DISCLAIMER:** Antes de continuar con el procedimiento, es necesario reiniciar el entorno de ejecución para evitar errores y obtener un proceso más fluido y uniforme.


Ahora bien, se procede a instalar la librería `Fairseq` para obtener los modelos de texto a voz de las bases de datos de la compañia Facebook.

In [2]:
!git clone https://github.com/pytorch/fairseq

fatal: destination path 'fairseq' already exists and is not an empty directory.


In [3]:
cd fairseq

/content/fairseq


In [4]:
pip install --editable ./

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92 kB 5.5 MB/s 
     |████████████████████████████████| 236 kB 15.6 MB/s 
     |████████████████████████████████| 123 kB 57.5 MB/s 
     |████████████████████████████████| 112 kB 60.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=61e2c476d81ae7f3ffe63b182365539dfa06e5c8352ceed4cb86950c8a6f57f5
  Stored in directory: /root/.cache/pip/wheels/ca/33/b7/336836125fc9bb4ceaa4376d8abca10ca8bc84ddc824baea6c
Successfully built antlr4-python3-runtime
  Running setup.py develop for fairseq


A continuación, se importan las librerías necesarias para el desarrollo del ejercicio propuesto. 

In [5]:
from huggingsound import SpeechRecognitionModel
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface

## **Parte 1.**

En esta sección, se dará una explicación detallada del proceso realzado para que a partir de una entrada de audio se reproduzca una grabación de su respectiva traducción. Con este proposito se desarrolla el código en tres partes: **Voz a Texto**, **Traducción de Texto** y **Texto a Voz**

### **Transición de Voz a Texto.**

Se determinan los modelos de transcripción para realizar el reconocimiento de voz y su transición a texto. 

In [ ]:
model_stt_es = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-spanish")
model_stt_en = pipeline("automatic-speech-recognition")

05/25/2022 01:10:31 - INFO - huggingsound.speech_recognition.model - Loading model...


Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/380 [00:00<?, ?B/s]

...

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

### **Traducción de Texto**

A continuación se establecen los modelos encargados de realizar la traducción de un idioma a otro. En este caso en particular, se optó por tomar modelos que no son multilenguaje, sino que manejan una traducción específica. En primer lugar, se tiene el modelo de Español a Inglés.

In [ ]:
tokenizer_es_en = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model_es_en = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Posteriormente, se seleccionó un modelo de traducción de Español a Francés.

In [ ]:
tokenizer_es_fr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-fr")
model_es_fr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-fr")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/800k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/793k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/317M [00:00<?, ?B/s]

De igual forma, se consideró el modelo de traducción de texto del idioma Inglés al Español.

In [ ]:
tokenizer_en_es = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model_en_es = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Para finalizar, se optó por tomar un modelo de traducción de Español a Ruso con el fin de verificar realmente la precisión que puede llegar a tener el código desarrollado.

In [ ]:
tokenizer_es_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-ru")
model_es_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-ru")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/810k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:196: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Downloading:   0%|          | 0.00/295M [00:00<?, ?B/s]

Se tomó la decisión de seleccionar estos modelos en particular con el fin de agregarle mayor versatilidad al ejercicio, esto además nos permite obtener varias medidas de comparación entre modelos y destacar los comportamientos satisfactorios. 

### **Transición de Texto a Voz**

Finalmente, se consideran los modelos encargados de realizar el cambio de Texto a Voz, donde el resultado debe ser una muestra de audio con la traducción en el idioma correspondiente de la muestra ingresada orginalmente. Dado que en la sección anterior se tomaron modelos muy específicos es necesario considerar los mismos idiomas en esta parte del proceso con el fin de obtener un resultado coherente. En primer lugar, en el idioma Inglés.

In [ ]:
models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False, 'cpu':True}
)

model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(models, cfg)

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/306 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/602 [00:00<?, ?B/s]

A continuación, se realiza el procedimiento anterior para los idiomas Francés, Ruso y Español, respectivamente.

In [ ]:
models_fr, cfg_fr, task_fr = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-fr-cv7_css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False, 'cpu':True}
)
model_fr = models_fr[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg_fr, task_fr.data_cfg)
generator_fr = task_fr.build_generator(models_fr, cfg_fr)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/610 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/395 [00:00<?, ?B/s]

In [ ]:
models_ru, cfg_ru, task_ru = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-ru-cv7_css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False, 'cpu':True}
)
model_ru = models_ru[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg_ru, task_ru.data_cfg)
generator_ru = task.build_generator(models_ru, cfg_ru)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/610 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/395 [00:00<?, ?B/s]

In [ ]:
models_es, cfg_es, task_es = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False, 'cpu':True}
)
model_es = models_es[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg_es, task_es.data_cfg)
generator_es = task.build_generator(models_es, cfg_es)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/55.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

## **Parte 2: Pipeline**

A continuación se realizará una prueba con una muestra de audio predeterminada en Español.

In [9]:
cd ..

/content


Se lee la muestra de audio correspondiente.

In [10]:
text = model_stt_es.transcribe(['prueba.ogg'])[0]['transcription']

100%|██████████| 1/1 [00:06<00:00,  6.62s/it]


Luego, se ilustra el resultado de proceso de transición de Voz a Texto.

In [12]:
text

'hola este es un audio de prueba para la materia introducción al procesamiento del lenguaje natural'

## **Parte 3: Multi-lenguaje**

Para continuar con el proceso, se realiza la traducción de Inglés a Español empleando los modelos de la sección anterior.


In [17]:
translated = model_es_en.generate(**tokenizer_es_en(text, return_tensors="pt", padding=True))
for t in translated:
  output_text = tokenizer_es_en.decode(t, skip_special_tokens=True) 

Y posteriormente, se visualiza el resultado de la traducción realizada.

In [18]:
output_text

'hello this is a test audio for the introduction matter to natural language processing'

Ahora bien, con el fin de probar otro modelo de traducción, se tomó de Inglés a Español y se realizó un proceso similar.

In [19]:
translated = model_en_es.generate(**tokenizer_en_es(output_text, return_tensors="pt", padding=True))
for t in translated:
  output_text_2 = tokenizer_en_es.decode(t, skip_special_tokens=True)

Se visualiza el texto plano después del proceso de traducción. 

In [21]:
output_text_2

'hola este es un audio de prueba para la materia de introducción al procesamiento del lenguaje natural'

Finalmente, se determina la predicción del modelo correspondiente y se presentan las muestras de audio puntuales, empleando modelos de Texto a Voz.

In [20]:
import IPython.display as ipd

In [22]:
sample = TTSHubInterface.get_model_input(task, output_text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [23]:
sample = TTSHubInterface.get_model_input(task_es, output_text_2)
wav, rate = TTSHubInterface.get_prediction(task_es, model_es, generator_es, sample)

ipd.Audio(wav, rate=rate)

## **Interfaz Gráfica.**

Con respecto a las politicas de privacidad de Colab y el tema de la necesidad de obtener permisos del microfono para obtener una muestra de audio directa, se debe ingresar al enlace externo de gradio para evidenciar la funcionalidad del programa. En este caso, se define la siguiente función que sintetiza el proceso realizado anteriormente, pero ya de forma más general:

$\quad$

--------------------------------------------------------------------------
### **Translate**
--------------------------------------------------------------------------
**INPUT:** Muestra de audio inicial (`audio`) junto con el idioma seleccionado (`language`).

- **Paso I:** En caso de haber recibido una muestra de audio nula, los resultados se ajustan con el valor `None`.

- **Paso II:** Se procede a identificar el idioma que predomina en al muestra de audio original, este puede ser Inglés o Español, en consistencia con los modelos explorados en la sección anterior. Posteriormente, se realiza la transición de Voz a Texto y el reaultado se alamcena en una variable (`text`).

- **Paso III:** Dependiendo de la traducción seleccionada en la interfaz (`Spanish to English`, `Spanish to French`, `Spanish to Russian` o `English to Spanish`), se procede a realizar la traducción deseada (`translated`), luego se decodifica cada una de las partes de la traducción (`output_text`), se determina la muestra (`sample`) que permite adaptar los datos de entrada a una representación que sea procesable para el modelo y finalmente, se obtiene la predicción del respectivo modelo (`wav`, `rate`). Este proceso se realiza de igual forma para cada una de las opciones de traducción ofrecidas inicialmente. 

**OUTPUT:** Una tupla que contiene la frecuencia de sonido (`rate`) junto con la señal de audio despues de pasar por el proceso de traducción (`wav`) en formato de `array` de la libreria `Numpy`, Adicionalmente, se proporciona el texto correspondiente a la traducción realizada (`output_text`).

In [ ]:
import gradio as gr
import time

def translate(audio, language):
    if audio == None:
        return None, None
    
    if language[:language.find("to")-1] == "Spanish":
      text = model_stt_es.transcribe([audio])[0]['transcription']
    elif language[:language.find("to")-1] == "English":
      text = model_stt_en(audio)["text"]

    if language == "Spanish to English":
      translated = model_es_en.generate(**tokenizer_es_en(text, return_tensors="pt", padding=True))
      for t in translated:
        output_text = tokenizer_es_en.decode(t, skip_special_tokens=True) 
      sample = TTSHubInterface.get_model_input(task, output_text)
      wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

    elif language == "Spanish to French":
      translated = model_es_fr.generate(**tokenizer_es_fr(text, return_tensors="pt", padding=True))
      for t in translated:
        output_text = tokenizer_es_fr.decode(t, skip_special_tokens=True) 
      sample = TTSHubInterface.get_model_input(task_fr, output_text)
      wav, rate = TTSHubInterface.get_prediction(task_fr, model_fr, generator_fr, sample)

    elif language == "Spanish to Russian":
      translated = model_es_ru.generate(**tokenizer_es_ru(text, return_tensors="pt", padding=True))
      for t in translated:
        output_text = tokenizer_es_ru.decode(t, skip_special_tokens=True)
      sample = TTSHubInterface.get_model_input(task_ru, output_text)
      wav, rate = TTSHubInterface.get_prediction(task_ru, model_ru, generator_ru, sample)

    elif language == "English to Spanish":
      translated = model_en_es.generate(**tokenizer_en_es(text, return_tensors="pt", padding=True))
      for t in translated:
        output_text = tokenizer_en_es.decode(t, skip_special_tokens=True)
      sample = TTSHubInterface.get_model_input(task_es, output_text)
      wav, rate = TTSHubInterface.get_prediction(task_es, model_es, generator_es, sample)
      
    return (rate, wav.numpy()), output_text

gr.Interface(
    fn=translate, 
    inputs= [gr.inputs.Audio(source="microphone", type="filepath"),
             gr.inputs.Dropdown(["Spanish to English", "Spanish to French", "Spanish to Russian", "English to Spanish"])],
    outputs=["audio","text"],
    live=True,
    interpretation="default").launch(debug = True)

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Hint: Set streaming=True for Audio component to use live streaming.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
Running on public URL: https://29974.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


100%|██████████| 1/1 [00:06<00:00,  6.58s/it]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


...

/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
100%|██████████| 1/1 [00:08<00:00,  8.35s/it]


05/18/2022 16:00:45 - WARNING - phonemizer - words count mismatch on 100.0% of the lines (1/1)


/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."
/usr/local/lib/python3.7/dist-packages/gradio/processing_utils.py:162: UserWarning: Audio data is not in 16-bit integer format.Trying to convert to 16-bit int format.
  "Audio data is not in 16-bit integer format."


Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7fe8ddc62c90>,
 'http://127.0.0.1:7860/',
 'https://29974.gradio.app')

Debido al trabajo realizado en esta sección, es posible destacar que el resultado que presenta un comportamiento adecuado y consistente es cuando se trabaja en el Inglés, lo cual tiene sentido considerando que estos modelos tienden a estar mejor balanceados, entrenados y con una cantidad de muestras considerable con respecto a los demás. El caso contrario se presenta al emplear el idioma Ruso, pues estos modelos no son tan robustos y completos en comparación, lo que lleva a obtener un resultado enredado y con varios fallos de sonido.

# **Conclusiones**

A partir del proceso realizado previamente, es posible destacar las siguientes conclusiones:

- Los modelos que ya han pasado por un proceso previo de entrenamiento de `HuggingFace` representan una herramienta poderosa con el fin de realizar el reconocimiento de voz y su transición a texto, debido seguramente a los procesos rigurosos de entrenamiento a los que algunos de ellos son sometidos. Esto se debe a que de por si ya se encuentran en un estado óptimo, pues tienen diversa variedad de parámetros y fueron entrenados con una cantidad considerable de datos, lo cual otorga una mejora de gran magnitud. Sin embargo, esto no implica que todos los modelos proporcionen un resultado óptimo.

- Con base en la implementación realizada, es posible evidenciar que los modelos encargados de realizar el proceso de Voz a Texto ofrezcan un resultado satisfactorio, al igual que los modelos que desarrollan el cambio de idioma, esto se debe seguramente a que la complejidad de estas tareas es tolerable y un buen desempeño es incluso accequible para la máquina en ejecución. Por otro lado, los algoritmos que realizan la transición de Texto a Voz presenta ciertas discrepancias e irregularidades al momento de tomar en cuenta acentos, tono y timbre que son variables más dificiles de entender, procesar y reproducir para un computador.

- Otra particularidad pertinente al estudio realizado, es que los modelos no cuentan con una cantidad infinita de recursos, es decir, internamente estandarizan las longitudes de cadenas de texto admitidas, esto con el fin de generalizar aún más el proceso de transición. Dependiendo de la magnitud de la muestra se realiza un recorte o un *padding* para ciertos modelos como el de Francés y Ruso. De forma que, si la muestra es una palabra o frase corta se termina con un resultado redundante, repetitivo e incluso incoherente. De lo contrario, si se presenta una muestra muy extensa tiende a realizar un recorte que toma por lo general la parte final de la cadena de texto. 

- Para finalizar, cabe destacar que no fue posible realizar la traducción en tiempo real, debido a cuestiones de la interfaz gradio e incluso particularidades del modelo mismo. Además, no tiene mucho sentido realizar traducción por secciones o pedazos conforme se recibe una muestra de audio puesto que por temas gramaticales y semánticos la muestra puede no cobrar sentido hasta el final de la misma. 

<div style="text-align: right"><b> Ana Valentina López Chacón & Isabella Martínez Martínez & Sara Julieth Zuleta Quevedo. </b></div>